* 请在环境变量中设置`DB_URI`指向数据库

In [1]:
import os
import numpy as np
import pandas as pd
from cvxpy import *
from cvxopt import *
from alphamind.api import *
from alphamind.portfolio.optimizers import QuadraticOptimizer

# Data Preparing
--------------------------

In [2]:
risk_penlty = 0.5
ref_date = '2020-02-21'
factor_name = "EMA5D"

engine = SqlEngine(os.environ['DB_URI'])
universe = Universe('hs300')
codes = engine.fetch_codes(ref_date, universe)

risk_cov, risk_exposure = engine.fetch_risk_model(ref_date, codes)
factor = engine.fetch_factor(ref_date, factor_name, codes)

total_data = pd.merge(factor, risk_exposure, on='code').dropna()
all_styles = risk_styles + industry_styles + macro_styles

risk_exposure_values = total_data[all_styles].values.astype(float)
special_risk_values = total_data['srisk'].values.astype(float)
risk_cov_values = risk_cov[all_styles].values

sec_cov_values_full = risk_exposure_values @ risk_cov_values @ risk_exposure_values.T / 10000  + np.diag(special_risk_values ** 2) / 10000
signal_full = total_data[factor_name].values

In [3]:
n = 200

sec_cov_values = sec_cov_values_full[:n, :n]
signal = signal_full[:n]

# Optimizing Weights
-------------------------------------

In [4]:
%%time
w = Variable(n)

lbound = 0.
ubound = 1. / n * 20

risk = sum_squares(multiply(special_risk_values[:n] / 100., w)) + quad_form((w.T @ risk_exposure_values[:n]).T, risk_cov_values[:n, :n] / 10000.)

objective = Minimize(risk_penlty * risk  - signal @ w)
constraints = [w >= lbound,
               w <= ubound,
               sum(w) == 1,]

prob = Problem(objective, constraints)

Wall time: 1 ms


In [5]:
%%time
prob.solve(verbose=True)

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 439, constraints m = 640
          nnz(P) + nnz(A) = 4419
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -7.8878e+03   4.61e+00   6.68e+04   1.00e-01   1.47e-03s
 125  -2.4830e+02   3.58e-07   2.76e-05   5.82e-01   5.10e-03s

status:               solved
solution polis

-248.2989412453118

In [6]:
prob.status, prob.value

('optimal', -248.2989412453118)

In [7]:
%%time
prob.solve(verbose=True, solver='ECOS')


ECOS 2.0.7 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  -2.156e+01  -1.165e+04  +2e+05  9e-01  2e-02  1e+00  6e+02    ---    ---    1  1  - |  -  - 
 1  +3.001e+02  -4.385e+03  +2e+05  3e-01  6e-03  2e+01  6e+02  0.1191  5e-01   2  2  2 |  0  0
 2  -2.457e+03  -3.271e+04  +2e+05  2e+00  4e-02  2e+02  6e+02  0.0130  1e+00   2  2  3 |  0  0
 3  -7.245e+02  -3.200e+03  +2e+05  3e-01  5e-03  1e+03  6e+02  0.1239  9e-01   2  2  2 |  0  0
 4  -3.144e+02  -5.230e+02  +6e+04  4e-02  5e-04  2e+02  2e+02  0.8185  2e-01   2  2  2 |  0  0
 5  -2.573e+02  -2.730e+02  +5e+03  3e-03  4e-05  2e+01  2e+01  0.9090  3e-03   1  2  2 |  0  0
 6  -2.493e+02  -2.522e+02  +8e+02  5e-04  6e-06  3e+00  3e+00  0.8565  2e-02   1  2  2 |  0  0
 7  -2.483e+02  -2.485e+02  +3e+01  2e-05  2e-07  4e-02  8e-02  0.9855  2e-02   1  2  2 |  0  0
 8  -2.483e+02  -2.483e+02  +2e+00  1e-06  1e-

-248.29896142281697

In [8]:
prob.status, prob.value

('optimal', -248.29896142281697)

In [9]:
%%time
P = matrix(sec_cov_values)
q = -matrix(signal)

G = np.zeros((2*n, n))
h = np.zeros(2*n)
for i in range(n):
    G[i, i] = 1.
    h[i] = 1. / n * 20
    G[i+n, i] = -1.
    h[i+n] = 0.
    
G = matrix(G)
h = matrix(h)
    
A = np.ones((1, n))
b = np.ones(1)

A = matrix(A)
b = matrix(b)

sol = solvers.qp(P, q, G, h, A, b)

     pcost       dcost       gap    pres   dres
 0: -6.5797e+05 -3.3860e+04  1e+08  7e+03  6e-16
 1: -1.7446e+04 -1.0601e+04  2e+06  1e+02  1e-13
 2: -4.0149e+02 -1.0036e+04  3e+04  2e+00  2e-13
 3: -1.8099e+02 -3.2793e+03  3e+03  2e-15  4e-14
 4: -1.9534e+02 -7.5624e+02  6e+02  2e-16  6e-15
 5: -2.1878e+02 -3.6280e+02  1e+02  2e-16  6e-16
 6: -2.2458e+02 -3.2078e+02  1e+02  2e-16  4e-16
 7: -2.3361e+02 -2.9012e+02  6e+01  2e-16  4e-16
 8: -2.3634e+02 -2.7977e+02  4e+01  2e-16  3e-16
 9: -2.3759e+02 -2.6601e+02  3e+01  2e-16  4e-16
10: -2.4290e+02 -2.5586e+02  1e+01  2e-16  3e-16
11: -2.4774e+02 -2.4901e+02  1e+00  2e-16  5e-16
12: -2.4829e+02 -2.4831e+02  1e-02  2e-16  4e-16
13: -2.4830e+02 -2.4830e+02  1e-04  2e-16  7e-16
Optimal solution found.
Wall time: 58 ms


In [10]:
%%time
lbound = np.zeros(n)
ubound = np.ones(n) * 20 / n
cons_matrix = np.ones((1, n))
clb = np.ones(1)
cub = np.ones(1)

cons_matrix = np.concatenate([cons_matrix, clb.reshape((-1, 1)), cub.reshape((-1, 1))], axis=1)
qpopt = QuadraticOptimizer(objective=-signal,
                           cons_matrix=cons_matrix,
                           lbound=lbound,
                           ubound=ubound,
                           factor_cov=risk_cov_values[:n, :n] / 10000.,
                           factor_load=risk_exposure_values[:n],
                           factor_special=special_risk_values[:n] * special_risk_values[:n] / 10000.)
print(qpopt.feval())
print(qpopt.status())

-248.29896142298423
optimal
Wall time: 39 ms


# Performace Timing
-------------------------

In [11]:
import datetime as dt

In [12]:
def time_function(py_callable, n):
    start = dt.datetime.now()
    val = py_callable(n)
    return (dt.datetime.now() - start).total_seconds(), val

In [13]:
def cvxpy(n):
    w = Variable(n)

    lbound = 0.
    ubound = 0.01
    
    risk = sum_squares(multiply(special_risk_values[:n] / 100., w)) + quad_form((w.T @ risk_exposure_values[:n]).T, risk_cov_values[:n, :n] / 10000.)

    objective = Minimize(risk_penlty * risk  - signal @ w)
    constraints = [w >= lbound,
                   w <= ubound,
                   sum(w) == 1,]

    prob = Problem(objective, constraints)
    prob.solve(verbose=False, solver='ECOS')
    return prob.value

In [14]:
def cvxopt(n):
    P = matrix(sec_cov_values)
    q = -matrix(signal)

    G = np.zeros((2*n, n))
    h = np.zeros(2*n)
    for i in range(n):
        G[i, i] = 1.
        h[i] = 0.01
        G[i+n, i] = -1.
        h[i+n] = 0.

    G = matrix(G)
    h = matrix(h)

    A = np.ones((1, n))
    b = np.ones(1)

    A = matrix(A)
    b = matrix(b)
    
    solvers.options['show_progress'] = False
    sol = solvers.qp(P, q, G, h, A, b)
    return sol['primal objective']

In [15]:
def alpha_mind(n):
    lbound = np.zeros(n)
    ubound = np.ones(n) * 0.01
    cons_matrix = np.ones((1, n))
    clb = np.ones(1)
    cub = np.ones(1)
    cons_matrix = np.concatenate([cons_matrix, clb.reshape((-1, 1)), cub.reshape((-1, 1))], axis=1)
    qpopt = QuadraticOptimizer(objective=-signal,
                               cons_matrix=cons_matrix,
                               lbound=lbound,
                               ubound=ubound,
                               factor_cov=risk_cov_values[:n, :n] / 10000.,
                               factor_load=risk_exposure_values[:n],
                               factor_special=special_risk_values[:n] * special_risk_values[:n] / 10000.)
    return qpopt.feval()

In [18]:
n_steps = list(range(100, 301, 100))
cvxpy_times = [None] * len(n_steps)
cvxopt_times = [None] * len(n_steps)
alpha_mind_times = [None] * len(n_steps)
print("{0:<8}{1:>12}{2:>12}{3:>12}".format('Scale(n)', 'cvxpy', 'cvxopt', 'alpha_mind'))

for i, n in enumerate(n_steps):
    sec_cov_values = sec_cov_values_full[:n, :n]
    signal = signal_full[:n]
    cvxpy_times[i], val1 = time_function(cvxpy, n)
    cvxopt_times[i], val2 = time_function(cvxopt, n)
    alpha_mind_times[i], val3 = time_function(alpha_mind, n)
    
    np.testing.assert_almost_equal(val1, val2, 4)
    np.testing.assert_almost_equal(val2, val3, 4)
    
    print("{0:<8}{1:>12.4f}{2:>12.4f}{3:>12.4f}".format(n, cvxpy_times[i], cvxopt_times[i], alpha_mind_times[i]))

Scale(n)       cvxpy      cvxopt  alpha_mind
100           0.0270      0.0260      0.0280
200           0.0330      0.0390      0.0340
300           0.0380      0.0610      0.0500
